In [2]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mea
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

## Wczytanie danych

In [3]:
df = pd.read_hdf("../data/car.h5")
df.sample()

/home/adrian/miniconda3/envs/dataworkshop/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,feature_wspomaganie-kierownicy,feature_podgrzewana-przednia-szyba,feature_przyciemniane-szyby,feature_elektrycznie-ustawiane-fotele,feature_klimatyzacja-czterostrefowa,feature_tuner-tv,feature_poduszki-boczne-przednie,feature_tempomat-aktywny,feature_klimatyzacja-automatyczna,feature_poduszki-boczne-tylne,feature_radio-fabryczne,feature_odtwarzacz-dvd,feature_czujniki-parkowania-tylne,feature_łopatki-zmiany-biegów,feature_centralny-zamek,feature_dach-panoramiczny,feature_kamera-cofania,feature_immobilizer,feature_czujnik-martwego-pola,feature_gniazdo-sd,feature_podgrzewane-lusterka-boczne,feature_podgrzewane-przednie-siedzenia,feature_hak,feature_abs,feature_wielofunkcyjna-kierownica,feature_światła-xenonowe,feature_gniazdo-usb,feature_tapicerka-skórzana,feature_gniazdo-aux,feature_esp-(stabilizacja-toru-jazdy),feature_alarm,feature_podgrzewane-tylne-siedzenia,feature_mp3,feature_tempomat,feature_hud-(wyświetlacz-przezierny),feature_elektrochromatyczne-lusterko-wsteczne,feature_relingi-dachowe,feature_elektrochromatyczne-lusterka-boczne,param_liczba-miejsc,param_uszkodzony,param_marka-pojazdu,param_model,param_engine-code,param_liczba-drzwi,param_first-registration,param_country-of-origin,param_pierwsza-rejestracja,param_gearbox,param_przebieg,param_faktura-vat,param_rok-produkcji,param_vin,param_perłowy,param_homologacja-ciężarowa,param_service-record,param_metallic,param_leasing-concession,param_color,param_financing-option,param_original-owner,param_vat-marża,param_kategoria,param_co2-emissions,param_leasing,param_mileage,param_zarejestrowany-jako-zabytek,param_napęd,param_wartość-wykupu,param_typ,param_metalik,param_pierwszy-właściciel,param_skrzynia-biegów,param_vat-discount,param_no-accident,param_liczba-pozostałych-rat,param_particle-filter,param_zarejestrowany-w-polsce,param_oferta-od,param_moc,param_make,param_kraj-pochodzenia,param_engine-power,param_year,param_serwisowany-w-aso,param_rodzaj-paliwa,param_możliwość-finansowania,param_registered-in-poland,param_transmission,param_miesięczna-rata,param_kod-silnika,param_version,param_opłata-początkowa,param_model-pojazdu,param_pojemność-skokowa,param_kierownica-po-prawej-(anglik),param_fuel-type,param_vat-free,param_acrylic,param_damaged,param_door-count,param_tuning,param_filtr-cząstek-stałych,param_nr-of-seats,param_kolor,param_engine-capacity,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
79267,"[Osobowe, Renault, Scenic, Renault Scenic II (...","14:55, 1 marca 2018",PLN,"Cena Brutto, Do negocjacji",6900.0,"ul. Antoniego Abrahama 19 - 84-300 Lębork, lęb...",Bauer,Dealer,False,False,False,False,False,False,True,False,True,True,True,True,True,True,True,False,False,True,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,True,True,True,False,False,False,True,False,False,False,False,False,False,False

## Feature Engineering

In [4]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
    if isinstance(df[feat][0], list):
        continue
    
    factorized_values = df[feat].factorize()[0]
    if SUFFIX_CAT in feat:
        df[feat] = factorized_values
    else:
        df[feat+ SUFFIX_CAT] = factorized_values

In [5]:
cat_feats =  [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

In [10]:
def run_model(model, feats):
    X = df[feats].values
    y = df['price_value'].values

    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
    return np.mean(scores), np.std(scores)

In [7]:
m =  DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2570 ± 0.0084,param_napęd__cat
0.2033 ± 0.0037,param_stan__cat
0.1957 ± 0.0021,param_faktura-vat__cat
0.1447 ± 0.0075,param_rok-produkcji__cat
0.0629 ± 0.0035,param_moc__cat
0.0415 ± 0.0012,feature_kamera-cofania__cat
0.0384 ± 0.0023,param_skrzynia-biegów__cat
0.0255 ± 0.0038,param_marka-pojazdu__cat
0.0196 ± 0.0014,param_pojemność-skokowa__cat
0.0162 ± 0.0003,feature_bluetooth__cat


## Decision Tree

In [11]:
model = DecisionTreeRegressor(max_depth=5)
run_model(model, cat_feats)

(-19695.13091100928, 148.72570644015937)

## Random Forest

In [12]:
run_model(RandomForestRegressor(max_depth=5, n_estimators = 100, random_state=0), cat_feats)

(-18719.72987663565, 97.5766154312288)

## XGBoost

In [31]:
xgb_params = {
    'max_depth':5,
    'n_estimatords':50,
    'learning_rate':0.1,
    'seed':0,
    'nthread': 3
    
}

In [33]:
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[15:08:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:08:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:08:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11862.434165529698, 264.2411021793361)

In [21]:
m =  xgb.XGBRegressor(**xgb_params)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[14:48:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1341 ± 0.0029,param_rok-produkcji__cat
0.1037 ± 0.0028,param_stan__cat
0.0956 ± 0.0029,param_napęd__cat
0.0501 ± 0.0020,param_skrzynia-biegów__cat
0.0442 ± 0.0014,param_moc__cat
0.0438 ± 0.0018,param_faktura-vat__cat
0.0401 ± 0.0008,param_marka-pojazdu__cat
0.0216 ± 0.0007,param_typ__cat
0.0182 ± 0.0012,feature_kamera-cofania__cat
0.0175 ± 0.0006,param_wersja__cat


In [28]:
feats = [
    'param_rok-produkcji__cat',
    'param_stan__cat',
    'param_napęd__cat',
    'param_skrzynia-biegów__cat',
    'param_moc__cat',
    'param_faktura-vat__cat',
    'param_marka-pojazdu__cat',
    'param_typ__cat', 
    'feature_kamera-cofania__cat',
    'param_wersja__cat',
    'param_model-pojazdu__cat',
    'param_pojemność-skokowa__cat',
    'param_kod-silnika__cat',
    'seller_name__cat',
    'feature_wspomaganie-kierownicy__cat',
    'feature_czujniki-parkowania-przednie__cat',
    'param_uszkodzony__cat',
    'feature_system-start-stop__cat',
    'feature_regulowane-zawieszenie__cat',
    'feature_asystent-pasa-ruchu__cat',
]

In [29]:
len(feats)

20

In [32]:
model =  xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[15:06:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11861.928807244389, 100.50210848351769)

In [34]:
df['param_napęd__cat']

0        -1
2         0
3         1
4         0
5         1
6         0
7         1
10        0
11        0
13        0
16        0
17        0
19        1
20        0
22        1
23        1
24        1
26        0
27        0
28        0
29        2
30        0
32        0
34        1
37        0
39        0
40        0
41        0
42        0
44        3
         ..
160573    0
160574    3
160575   -1
160576   -1
160577    0
160578    4
160579   -1
160580    0
160582   -1
160583    4
160584    2
160585    0
160588    0
160590   -1
160591    0
160592   -1
160593    0
160595   -1
160596   -1
160599    0
160601    0
160604    2
160605    0
160606    0
160608    0
160609    0
160610    0
160611    0
160614    1
160615    0
Name: param_napęd__cat, Length: 106494, dtype: int64

In [36]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [37]:
feats = [
    'param_rok-produkcji',
    'param_stan__cat',
    'param_napęd__cat',
    'param_skrzynia-biegów__cat',
    'param_moc__cat',
    'param_faktura-vat__cat',
    'param_marka-pojazdu__cat',
    'param_typ__cat', 
    'feature_kamera-cofania__cat',
    'param_wersja__cat',
    'param_model-pojazdu__cat',
    'param_pojemność-skokowa__cat',
    'param_kod-silnika__cat',
    'seller_name__cat',
    'feature_wspomaganie-kierownicy__cat',
    'feature_czujniki-parkowania-przednie__cat',
    'param_uszkodzony__cat',
    'feature_system-start-stop__cat',
    'feature_regulowane-zawieszenie__cat',
    'feature_asystent-pasa-ruchu__cat',
]

In [38]:
model =  xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[17:33:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:33:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:33:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10067.086619567794, 35.57616233370741)

In [40]:
df['param_moc'].unique()

array(['90 KM', '115 KM', '262 KM', '110 KM', '310 KM', '105 KM',
       '140 KM', '175 KM', '125 KM', '185 KM', '190 KM', '440 KM',
       '141 KM', '200 KM', '224 KM', '75 KM', '99 KM', '184 KM', '109 KM',
       '233 KM', '116 KM', '68 KM', '286 KM', '126 KM', '160 KM',
       '135 KM', '120 KM', '272 KM', None, '150 KM', '180 KM', '136 KM',
       '102 KM', '131 KM', '218 KM', '245 KM', '170 KM', '112 KM',
       '250 KM', '252 KM', '73 KM', '100 KM', '313 KM', '101 KM',
       '285 KM', '70 KM', '383 KM', '174 KM', '277 KM', '132 KM',
       '130 KM', '215 KM', '60 KM', '330 KM', '163 KM', '177 KM', '98 KM',
       '78 KM', '189 KM', '156 KM', '143 KM', '69 KM', '113 KM', '65 KM',
       '122 KM', '82 KM', '251 KM', '95 KM', '197 KM', '235 KM', '238 KM',
       '171 KM', '381 KM', '400 KM', '178 KM', '80 KM', '165 KM', '85 KM',
       '258 KM', '142 KM', '204 KM', '124 KM', '55 KM', '144 KM',
       '231 KM', '248 KM', '152 KM', '181 KM', '210 KM', '340 KM',
       '129 KM', '147 

In [45]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) =='None' else int(x.split(' ')[0]))

In [46]:
feats = [
    'param_rok-produkcji',
    'param_stan__cat',
    'param_napęd__cat',
    'param_skrzynia-biegów__cat',
    'param_moc',
    'param_faktura-vat__cat',
    'param_marka-pojazdu__cat',
    'param_typ__cat', 
    'feature_kamera-cofania__cat',
    'param_wersja__cat',
    'param_model-pojazdu__cat',
    'param_pojemność-skokowa__cat',
    'param_kod-silnika__cat',
    'seller_name__cat',
    'feature_wspomaganie-kierownicy__cat',
    'feature_czujniki-parkowania-przednie__cat',
    'param_uszkodzony__cat',
    'feature_system-start-stop__cat',
    'feature_regulowane-zawieszenie__cat',
    'feature_asystent-pasa-ruchu__cat',
]

In [47]:
model =  xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[17:40:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:40:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:40:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8698.488115726954, 52.63025457895438)

In [52]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [59]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) =='None' else int(x.split('cm')[0].replace(' ', '')))

In [60]:
feats = [
    'param_rok-produkcji',
    'param_stan__cat',
    'param_napęd__cat',
    'param_skrzynia-biegów__cat',
    'param_moc',
    'param_faktura-vat__cat',
    'param_marka-pojazdu__cat',
    'param_typ__cat', 
    'feature_kamera-cofania__cat',
    'param_wersja__cat',
    'param_model-pojazdu__cat',
    'param_pojemność-skokowa',
    'param_kod-silnika__cat',
    'seller_name__cat',
    'feature_wspomaganie-kierownicy__cat',
    'feature_czujniki-parkowania-przednie__cat',
    'param_uszkodzony__cat',
    'feature_system-start-stop__cat',
    'feature_regulowane-zawieszenie__cat',
    'feature_asystent-pasa-ruchu__cat',
]

In [61]:
model =  xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[17:45:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:45:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:45:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8580.61650667904, 58.180956552556225)

In [62]:
df['param_kod-silnika'].unique()

array([None, '750', '330', 'CLS 350', '316', '525', 'C 200', 'C 270',
       'S 400', '335', 'CLA 220', '523', '320', '318', 'ML 320', 'A 180',
       '118', 'A 160', 'C 220', 'A45 AMG', 'B 180', 'GLE 350', 'C 180',
       '735', '116', '420', 'E 220', '730', '740', '530', 'CLA 45 AMG',
       'CLA 200', 'E 200', '535', 'V 250', '520', '428', '120', 'GLA 200',
       'ML 350', 'GLA 180', '518', 'CLK 200', 'E 240', 'SL 350', 'GL 320',
       '528', 'E 63 AMG', 'CLA 250', 'CLK 220', 'GLE 43 AMG', 'S 320',
       'S 500', 'ML 270', '540', '328', 'E 320', 'S 55 AMG', 'A 170',
       'R 320', 'CLK 270', 'E 350', 'GLE 63 AMG', 'ML 420', 'C 63 AMG',
       'SLK 200', 'SLK 320', 'S 63 AMG', '325', 'V 200', 'GLC 220',
       'E 300', 'C 350', 'E 280', 'SL 500', 'ML 63 AMG', 'B 200', '650',
       '745', 'GLA 250', 'B 150', 'CL 63 AMG', '5GT (F07)', 'E 270',
       'A 200', 'E 250', 'GLA 220', 'E 500', 'C 250', 'GL 500', 'CLK 320',
       'SL 63 AMG', 'B 250', 'A 190', 'ML 280', 'G 500', 'C 300'